# Janesick Breast Cancer Analysis

This notebook demonstrates the SPATCH analysis pipeline on the **Janesick et al. (Nature Communications, 2023)** FFPE human breast cancer sample.

**Dataset**: Xenium FFPE Human Breast Cancer Rep1  
**Sample**: Invasive Ductal Carcinoma with DCIS  
**Source**: https://github.com/10XGenomics/janesick_nature_comms_2023_companion

## Analysis Pipeline:
1. Load Xenium data using SpatialData
2. Quality control and preprocessing
3. Clustering and cell type annotation
4. Spatial analysis (neighborhoods, interactions)
5. SPATCH custom modules (cell shape, diffusion)
6. Visualization and interpretation

In [1]:
# Core libraries
import spatialdata as sd
import spatialdata_io as sdio
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd

# Custom SPATCH modules
from spatch_modules import get_module, list_modules, run_single_module

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configure
sc.settings.verbosity = 2
sc.settings.set_figure_params(dpi=100, facecolor='white', frameon=False)
sc.logging.print_header()

# Output directory
OUTPUT_DIR = "../results/janesick_breast_cancer/"
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/figures", exist_ok=True)

ModuleNotFoundError: No module named 'spatialdata'

## 1. Load Xenium Data

Load the Janesick breast cancer Xenium data using SpatialData-io.

In [ ]:
# Path to Xenium output directory
DATA_PATH = "../data/outs/"

# Load with spatialdata_io
print(f"Loading Xenium data from {DATA_PATH}...")
sdata = sdio.xenium(DATA_PATH)

print("\n" + "="*70)
print("SpatialData Object:")
print("="*70)
print(sdata)

In [ ]:
# Get the cell x gene table
adata = sdata.tables["table"]

print(f"\nAnnData shape: {adata.shape[0]} cells × {adata.shape[1]} genes")
print(f"Total transcripts: {adata.X.sum():.0f}")
print(f"Mean transcripts/cell: {adata.X.sum(axis=1).mean():.1f}")

## 2. Quality Control

Calculate QC metrics and visualize data quality.

In [ ]:
# Calculate QC metrics
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # Mitochondrial genes
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

print("\nQC Metrics Summary:")
print("="*70)
print(adata.obs[['total_counts', 'n_genes_by_counts', 'pct_counts_mt']].describe())

In [ ]:
# Visualize QC metrics
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Total counts
sc.pl.violin(adata, 'total_counts', ax=axes[0, 0], show=False)
axes[0, 0].set_title('Total transcript counts per cell')

# Gene counts
sc.pl.violin(adata, 'n_genes_by_counts', ax=axes[0, 1], show=False)
axes[0, 1].set_title('Number of genes per cell')

# Mitochondrial %
sc.pl.violin(adata, 'pct_counts_mt', ax=axes[1, 0], show=False)
axes[1, 0].set_title('Mitochondrial transcript %')

# Scatter: counts vs genes
axes[1, 1].scatter(adata.obs['total_counts'], adata.obs['n_genes_by_counts'], 
                   alpha=0.3, s=1)
axes[1, 1].set_xlabel('Total counts')
axes[1, 1].set_ylabel('Number of genes')
axes[1, 1].set_title('Counts vs Genes')

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/figures/qc_metrics.png", dpi=300, bbox_inches='tight')
plt.show()

## 3. Filtering and Normalization

In [ ]:
# Filter cells
print(f"Before filtering: {adata.n_obs} cells")

sc.pp.filter_cells(adata, min_genes=50)
sc.pp.filter_cells(adata, max_genes=10000)
adata = adata[adata.obs.pct_counts_mt < 25, :].copy()

print(f"After filtering: {adata.n_obs} cells")

# Filter genes
print(f"\nBefore filtering: {adata.n_vars} genes")
sc.pp.filter_genes(adata, min_cells=5)
print(f"After filtering: {adata.n_vars} genes")

In [ ]:
# Normalize and log-transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Store raw normalized data
adata.raw = adata

print("✓ Normalization complete")

## 4. Feature Selection and Dimensionality Reduction

In [ ]:
# Highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=3000, flavor='seurat_v3')

print(f"Highly variable genes: {adata.var['highly_variable'].sum()}")

# PCA
sc.pp.pca(adata, n_comps=50)

# Neighbors graph
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=50)

# UMAP
sc.tl.umap(adata, min_dist=0.3)

print("✓ Dimensionality reduction complete")

In [ ]:
# Visualize PCA variance ratio
sc.pl.pca_variance_ratio(adata, n_pcs=50, log=True)

## 5. Clustering

Perform multi-resolution Leiden clustering to capture tumor heterogeneity.

In [ ]:
# Leiden clustering at multiple resolutions
resolutions = [0.3, 0.5, 0.8, 1.0, 1.5]

for res in resolutions:
    sc.tl.leiden(adata, resolution=res, key_added=f'leiden_{res}')
    n_clusters = adata.obs[f'leiden_{res}'].nunique()
    print(f"Resolution {res}: {n_clusters} clusters")

In [ ]:
# Visualize clustering at different resolutions
sc.pl.umap(adata, color=[f'leiden_{res}' for res in resolutions], 
           ncols=3, wspace=0.3, legend_loc='on data', legend_fontsize=6)

## 6. Spatial Analysis

Compute spatial neighbors and visualize spatial structure.

In [ ]:
# Spatial neighbors graph
sq.gr.spatial_neighbors(adata, coord_type='generic', n_neighs=30)

print(f"✓ Spatial graph: {adata.n_obs} nodes, "
      f"{adata.obsp['spatial_connectivities'].nnz} edges")

In [ ]:
# Spatial visualization of clusters
sq.pl.spatial_scatter(
    adata, 
    color='leiden_0.8',
    size=2,
    figsize=(15, 15),
    title='Spatial distribution of clusters (resolution 0.8)'
)
plt.savefig(f"{OUTPUT_DIR}/figures/spatial_clusters.png", dpi=300, bbox_inches='tight')

## 7. Cell Type Annotation

Annotate cell types using marker genes specific to breast cancer microenvironment.

In [ ]:
# Define marker genes for breast cancer cell types
markers = {
    'Tumor_Invasive': ['FASN', 'CEACAM6', 'MKI67'],
    'Tumor_DCIS': ['CEACAM6', 'ESR1'],
    'Myoepithelial': ['ACTA2', 'KRT15', 'KRT14'],
    'Stromal': ['POSTN', 'COL1A1', 'DCN'],
    'T_cells': ['CD3E', 'CD3D', 'IL7R'],
    'Macrophages': ['ITGAX', 'CD68', 'LYZ'],
    'B_cells': ['MS4A1', 'CD79A'],
    'Endothelial': ['VWF', 'PECAM1']
}

# Score cells for each marker set
for cell_type, genes in markers.items():
    # Filter to genes present in data
    genes_present = [g for g in genes if g in adata.var_names]
    if genes_present:
        sc.tl.score_genes(adata, genes_present, score_name=f'{cell_type}_score')
        print(f"✓ Scored {cell_type}: {len(genes_present)}/{len(genes)} genes")

In [ ]:
# Visualize marker scores on UMAP
score_cols = [f'{ct}_score' for ct in markers.keys()]
sc.pl.umap(adata, color=score_cols, ncols=4, cmap='viridis', vmax='p99')

In [ ]:
# Spatial visualization of marker genes
marker_genes = ['FASN', 'CEACAM6', 'ACTA2', 'KRT15', 'POSTN', 'CD3E', 'ITGAX', 'VWF']
genes_present = [g for g in marker_genes if g in adata.var_names]

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, gene in enumerate(genes_present[:8]):
    sq.pl.spatial_scatter(
        adata, 
        color=gene,
        size=2,
        ax=axes[i],
        cmap='viridis',
        title=gene
    )

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/figures/spatial_markers.png", dpi=300, bbox_inches='tight')
plt.show()

## 8. SPATCH Custom Modules

Run SPATCH-specific analysis modules.

### 8.1 Cell Shape Metrics

In [ ]:
# Run cell shape metrics module
print("Running cell shape metrics analysis...")
sdata = run_single_module(
    sdata,
    "cell_shape_metrics",
    boundaries_key="cell_boundaries",
    table_key="table"
)

# Update adata reference
adata = sdata.tables["table"]

In [ ]:
# Visualize cell shape distributions
shape_metrics = ['area_um2', 'circularity', 'eccentricity', 'solidity']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, metric in enumerate(shape_metrics):
    if metric in adata.obs.columns:
        adata.obs[metric].hist(ax=axes[i], bins=50, edgecolor='black')
        axes[i].set_xlabel(metric)
        axes[i].set_ylabel('Frequency')
        axes[i].set_title(f'{metric} Distribution')

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/figures/cell_shape_distributions.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Cell shape by cluster
if 'area_um2' in adata.obs.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Area by cluster
    cluster_col = 'leiden_0.8'
    adata.obs.boxplot(column='area_um2', by=cluster_col, ax=axes[0], rot=45)
    axes[0].set_title('Cell Area by Cluster')
    axes[0].set_xlabel('Cluster')
    axes[0].set_ylabel('Area (µm²)')
    
    # Circularity by cluster
    adata.obs.boxplot(column='circularity', by=cluster_col, ax=axes[1], rot=45)
    axes[1].set_title('Cell Circularity by Cluster')
    axes[1].set_xlabel('Cluster')
    axes[1].set_ylabel('Circularity')
    
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/figures/cell_shape_by_cluster.png", dpi=300, bbox_inches='tight')
    plt.show()

### 8.2 Diffusion Analysis

In [ ]:
# Run diffusion analysis module
print("Running diffusion analysis...")
sdata = run_single_module(
    sdata,
    "diffusion_analysis",
    table_key="table",
    in_tissue_col="in_tissue",
    compute_distances=True
)

## 9. Spatial Statistics

In [ ]:
# Neighborhood enrichment analysis
sq.gr.nhood_enrichment(adata, cluster_key='leiden_0.8')
sq.pl.nhood_enrichment(adata, cluster_key='leiden_0.8', figsize=(8, 8))
plt.savefig(f"{OUTPUT_DIR}/figures/neighborhood_enrichment.png", dpi=300, bbox_inches='tight')

In [ ]:
# Co-occurrence analysis
sq.gr.co_occurrence(adata, cluster_key='leiden_0.8')
sq.pl.co_occurrence(
    adata, 
    cluster_key='leiden_0.8',
    clusters=['0', '1', '2', '3', '4'],  # Adjust based on your clusters
    figsize=(10, 4)
)
plt.savefig(f"{OUTPUT_DIR}/figures/cooccurrence.png", dpi=300, bbox_inches='tight')

## 10. Differential Expression

Find marker genes for each cluster.

In [ ]:
# Run differential expression
sc.tl.rank_genes_groups(adata, groupby='leiden_0.8', method='wilcoxon')

# Visualize top markers
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False)

In [ ]:
# Get marker gene table
markers_df = sc.get.rank_genes_groups_df(adata, group=None)
markers_df.head(50)

In [ ]:
# Heatmap of top markers
sc.pl.rank_genes_groups_heatmap(
    adata, 
    n_genes=10,
    groupby='leiden_0.8',
    cmap='viridis',
    figsize=(12, 10)
)
plt.savefig(f"{OUTPUT_DIR}/figures/marker_heatmap.png", dpi=300, bbox_inches='tight')

## 11. Save Results

In [ ]:
# Save processed SpatialData
print("Saving results...")
sdata.write(f"{OUTPUT_DIR}/processed.zarr")
print(f"✓ Saved SpatialData to {OUTPUT_DIR}/processed.zarr")

# Save AnnData table
adata.write(f"{OUTPUT_DIR}/processed_table.h5ad")
print(f"✓ Saved AnnData to {OUTPUT_DIR}/processed_table.h5ad")

# Save marker genes
markers_df.to_csv(f"{OUTPUT_DIR}/marker_genes.csv", index=False)
print(f"✓ Saved marker genes to {OUTPUT_DIR}/marker_genes.csv")

print("\n" + "="*70)
print("Analysis complete!")
print("="*70)

## 12. Summary Statistics

In [ ]:
# Print summary
print("\n" + "="*70)
print("ANALYSIS SUMMARY")
print("="*70)
print(f"Dataset: Janesick Breast Cancer (FFPE, Xenium)")
print(f"Cells analyzed: {adata.n_obs:,}")
print(f"Genes detected: {adata.n_vars:,}")
print(f"Total transcripts: {adata.X.sum():,.0f}")
print(f"\nClusters identified (res=0.8): {adata.obs['leiden_0.8'].nunique()}")
print(f"\nCluster sizes:")
print(adata.obs['leiden_0.8'].value_counts().sort_index())
print("\n" + "="*70)